# Pareto Archetypal Analysis of PBMCs among Infected Covid-19  Patients

Sars-Cov-2 not only has been shown to damage the respiratory tract and lung tissues, but it has a substantial effect on neurological pathologies, the cardiovascular system and thrombotic effects throughout different organs particularly organs with high abundance of ACE2 receptors. This strongly suggests the virus can move past the lungs and has a role within the endothelial cell linings of the blood vessels. Mechanisms behind the virus's invasive properties and ability to cause blood vessel and brain damage is explored to determine proteins that can be considered for drug targets to mitigate virus spread.

Archetypal analysis is an unsupervised learning method to identify archetypes or extreme points in multidimensional data. The archetypes are put on a Pareto front, that is, no state can be made better without making one worse(Pareto optimality) where these arcs are contained within an efficient representation of a polytope using the principal convex hull algorithm. In a biological context, archetypes found in gene expression data may indicate highly expressed genes, that is, certain tradeoffs are made within cell and genes to achieve an optimal biological function determined by the archetype.

Here, I implement a Pareto Archetypal Analysis using R's ParetoTI package to find key biological processes or archetypes within peripheral blood mononuclear cells(PBMC) of infected Covid-19 Patients. I use Python's gseapy package to perform a gene set enrichment analysis at each archetype, and observe protein-protein interactions(PPI) at each archetype using STRING. Another Archetypal Analysis is done on CD4 T cells determined to be the cell type closest to the specific archetype of interest along with a GSEA and PPI network analysis.

Single-cell metadata of PBMCs taken from the Human Covid-19 Cell Atlas.

The following steps in R include:

1. Reading in the RDS File containing PBMCs metadata

2. Subsetting/Filtering cells and genes by visualizing violin and correlation plots. Violin plots for mitochondrial contamination, gene counts, and molecule counts are shown to observe where most of the data lies. Correlating molecule count with mitochondria contamination and gene counts was done to subset my data; I include genes that have a strong correlation with the molecular count. Although I include quite a significant amount of mitochondrial contamination, most of the data lies between 2.5 and 10 percent contamination; a more strict threshold will not give me biological signal. However, I will regress this out when I scale my data later on.

3. Normalization, finding variable features, and scaling using Seurat functions. Percent of mitochondria is regressed out.

4. Principal Component Analysis and determining how many principal components to use based on a skree plot. 10 principal components were used as they contribute to the most variation.

5. Finding the optimal number of archetypes to use based on t-ratio and variation plots. Although I find that 8 archetypes contribute to most of the variation in my dataset, the 6th archetype has the highest t-ratio, that is the most signficant, therefore, 6 archetypes were used.

6. Run principal convex hull algorithm with 6 archetypes. 

7. I used other functions within ParetoTI to find enriched genes at each archetype using a Wilcoxon one versus all test where we use a Euclidean distance matrix for the features and find a decreasing function from each feature to the archetype. I adjust p values for multiple comparisons using the Benjamin-Hochberg procedure.

8. The enriched genes were written to a data table to be used for a gene set enrichment analysis to discover biological processes using Python’s gseapy package, in which a pre ranked GSEA was run where the test statistic for gene ranks was the Wilcoxon mean log ratio. This automatically puts gene rankeed by archetype distance in order.

# R code for the Pareto Archetype Analysis 


library(ParetoTI)

library(data.table)

pbmcs <- readRDS("C:\\Users\\zulfi\\Desktop\\blish_covid.seu.rds")

library(Seurat)

DefaultAssay(pbmcs) <- "RNA"

VlnPlot(pbmcs,features = c("nFeature_RNA","nCount_RNA","percent.mt"),ncol = 3)


plot1 <- FeatureScatter(pbmcs,feature1 = "nCount_RNA",feature2 = "percent.mt")

plot2 <- FeatureScatter(pbmcs, feature1 = "nCount_RNA", feature2 = "nFeature_ RNA")

plot1 + plot2

<img src="cmp.png" width="800" />

pbmcs <- subset(pbmcs, subset = nFeature_RNA < 4000 & nFeature_RNA > 500 & percent.mt < 10 & percent.mt > 2.5)

plot3 <- FeatureScatter(pbmcs, feature1 = "nCount_RNA", feature2 = "nFeature_ RNA")

print(plot3)

<img src="cmp2.png" width="800" />

covid <- subset(pbmcs,Status=="COVID")

memory.limit(30000)

covid <- NormalizeData(covid,normalization.method = "LogNormalize",scale.factor = 10000, assay = "RNA")

covid <- FindVariableFeatures(covid, selection.method = "vst", nfeatures = 2000,assay = "RNA")

covid <- ScaleData(covid,assay = "RNA",vars.to.regress = "percent.mt")

covid <- RunPCA(covid, features = VariableFeatures(object = covid),assay = "RNA",npcs = 50)

eigs <- covid@reductions$pca@stdev**2

props <- eigs/sum(eigs)

plot(props,ylab = "Proportion of Variation",xlab="Principal Components")

<img src="PCA.png" width="800" />


pca <- data.frame(covid@reductions$pca\@cell.embeddings)

pca <- pca[c(1:10)]

pca <- data.matrix(pca)

pca <- t(pca)

pcha <- k_fit_pch(pca,ks = 3:8,maxiter = 500,seed=345)

plot_arc_var(pcha, type = "varexpl", point_size = 2)

plot_arc_var(pcha, type = "t_ratio", point_size = 2)

<img src="var.png" width="800" />

<img src="trat.png" width="800" />


arc <- fit_pch(pca,noc= 6,conv_crit = 1e-06,maxiter = 500,volume_ratio= "t_ratio")

plt_arc <- plot_arc(arc_data = arc, data = pca)

print(plt_arc)


<img src="pbmcsArcs.png" width="800" />


mergeDat <- merge_arch_dist(arc_data = arc,data = pca,dist_metric = c("euclidean", "arch_weights")[1],feature_data = covid@assays$RNA\@scale.data)

enrGenes <- find_decreasing_wilcox(mergeDat\\$data,mergeDat\\$arc_col,
features= mergeDat$features_col,bin = 0.1,method = "BioQC")

genesAtArcs <- get_top_decreasing(summary_genes = enrGenes,summary_sets = NULL, cutoff_genes = 0.001,cutoff_metric = "wilcoxon_p_val",p.adjust.method = "BH")


# Perform a pre-ranked Gene Set Enrichment Analysis at each Archetype 

In [2]:
import gseapy
import pandas as pd

In [5]:
'''

This short script takes in a data table in a csv file and performs a preranked GSEA. I use the top n genes closest to the
archetype for the enrichment analysis and output the top 10 biological processes ranked by the normalized enrichment score;
the overrepresented upregulated processes are shown. 

The GSEA uses pre ranked gene lists with the top n genes and the wilcoxon mean log ratio as the test statistic. 

Class Attributes:

fileObj = Data table file of archetypes and corresponding enriched genes
nGenes = Number of genes to use within each archetype for the gene set enrichment analysis

'''

class ParetoGSEA:
    
    def __init__(self,fileObj,nGenes):
        
        '''
        Class Parameters:
        
        fileObj = csv file containing enriched genes at each arc, from ParetoTI 
        nGenes = number of genes to include for the enrichment analysis from each archetype
        
        '''
        
        self.fileObj = pd.read_csv(fileObj)
        self.nGenes = nGenes
        self.arcDict = {}
        
    def topGenesOnly(self):
        
        '''
        Returns a dictionary of panda dataframes consisting top nGenes and the log mean wilcoxon score for each
        archetype
        
        '''
    
        arches = self.fileObj.iloc[:, 0].unique()
              
        for arc in arches:
            
            eachArch = self.fileObj['arch_name'] == arc
                        
            topGenes = self.fileObj[eachArch][:self.nGenes]
            
            self.arcDict[arc] = pd.DataFrame(list(topGenes["mean_diff"]),list(topGenes["genes"]))
            
        return self.arcDict
        
    def runGSEA(self):
        
        '''Runs a pre-rank gene set enrichment analysis for each archetype'''
        
        for arc,data in self.arcDict.items():
            
            data.to_csv('rankfile')
            
            df = pd.read_csv('rankfile',header=None)[2:]
            
            results = gseapy.prerank(rnk = df,gene_sets='GO_Biological_Process_2018', processes=10,min_size = 1)
            
            print(arc)
            print(results.res2d.sort_values('nes', ascending=False).head(10))
            print('\n\n')
            
    def saveGeneRanks(self):
        
        '''Write gene and scores to a file for STRING network analysis'''
        
        mergeData = [ dt for dt in self.arcDict.values() ]
        
        mergeData = pd.concat(mergeData)
        
        mergeData.reset_index(level=0, inplace=True)

        mergeData.columns = ["protein","log-fold-change"]
        
        mergeData.to_csv("CD4Arcs.txt", sep="\t",index=False)

In [6]:
'''Run a pre-ranked gene set enrichment analysis on the results of a Pareto task inference done on all PBMCs.'''
gsea = ParetoGSEA(fileObj = 'covidArcs.csv',nGenes = 25)
gsea.topGenesOnly()
gsea.runGSEA()

archetype_1
                                                          es       nes  \
Term                                                                     
positive regulation of protein insertion into m...  0.956522  2.658289   
antimicrobial humoral immune response mediated ...  1.000000  2.650991   
positive regulation of establishment of protein...  0.956522  2.573099   
protein maturation (GO:0051604)                     0.818182  2.563989   
dendritic cell chemotaxis (GO:0002407)              0.869565  2.526848   
positive regulation of protein localization to ...  0.956522  2.522068   
granulocyte chemotaxis (GO:0071621)                 0.869565  2.518257   
positive regulation of mitochondrial outer memb...  0.956522  2.514861   
regulation of protein insertion into mitochondr...  0.956522  2.502559   
regulation of phosphatidylinositol 3-kinase sig...  0.869565  2.490040   

                                                        pval  fdr  \
Term                          

# STRING to create protein-protein interactions at each Archetype

Key biological processes occuring at each archetype will be determined by STRING's GO biological processes. I use a .80 interaction score and remove nodes that are disconnected. The networks are displayed by confidence seen by the strength(thickness and color) of the edge connecting nodes.

Interior node colors(NOT THE OUTER SHADE) are labeled for processes/function mentioned.

In [ ]:
'''Create a file with gene and scores to be used for a network analysis using STRING online tool.'''

gsea = ParetoGSEA(fileObj = 'covidArcs.csv',nGenes = 25)
gsea.topGenesOnly()
gsea.saveGeneRanks()

Archetype 1:

The main processes include immune and cellular defense response by granzymes facilitated by Perforin-1(PRF1). IL2RB also likely induces T cell signaling here.

Different from the expected defense responses, SYNE1, SYNE2 and CX3CR1 may contribute to dendritic cell chemotaxis and cerebral cortex migration(red) through actin filament binding(blue). These processes suggest brain cell protrusion by SarsCov2 and the ability for neuropathology. Drugs to inhibit premature formation of dendritic cells and actin filament polymerization may be necessary in the future as these would slow virus spread within the brain.

<img src="arc1.png" width="800" />

Archetype 2

ITGA2B and ITGB3 interaction brings about rapid platelet aggregation physically plugging rupturing endothelial cell surfaces.(STRING). HIST1H2AC role in transcriptional regulation and therefore gene activity may induce integrin family proteins at the site of the extracellular matrix to result in clotting effects; these events would suggest gene expression responses due to blood vessel damage. PF4, SPARC, and PF4V1 contribute to wound healing(blue) based on platelet activation(red) and contribute to platelet aggregation resulting in blood clots. These gene associations contribute to the vascular damage, given past research on SarsCov ability to enter blood vessel for widespread infection, SarsCov2 may not only have the same properties, but may cause microvascular damage in the process leading to blood coagulation(1). Inhibition of ITGA2B and ITGB3 to preven rapid coagulation through blood thinners may benefit infected patients with cardiovascular disease. 

1. https://www.frontiersin.org/articles/10.3389/fphar.2020.00836/full

<img src="arc2.png" width="800" />

Archetype 3

Genes in response to coagulation include SERPINA1,CLU,and VCAN. These genes relate to leukocyte differentiation, myeloid leukocyte activation, and white blood cell mediated immunity evident through CD molecule function, macrophage proteins, and neutrophil activation. 

S100A9, S100A8, and S100A12 contribute to inflammatory responses not only for leukocytes but for cytokines and chemokines and induce neutrophil chemotaxis through the ERK1/2 and MAPK dependent processes; these signaling processes stimulate inflammotory response and thus viral role can be assumed to impact these signaling pathways. Inhibiting ERK1 and MAPK pathways in cardiovascular patients may lessen overall aggravation to prevent an overdrive of a cytokine production.

<img src="arc3.png" width="800" />

Archetype 4

Genes at this arch contribute to mitotic cell cycle processes,cell division,and other cell cycle processes. Certain genes contribute to cell differentiation and growth including SHCBP1, an activator of fibroblast growth factor in neuronal progenitor cells and MYBL2. There may be significance in the CCNA2 to induce cyclin dependent reactions in cell differentiation. Furthermore, based on further research conducted by UCSF to study SarsCov2 dependency on kinases, the pathways may not suggest growth, but rather disfunction in these growth pathways. Further analysis of UCSFs findings by European Molecular Biology Laboratory’s European Biosciences Institute found 49 kinases with abnormal function including a network involving p38/MAPK pathways leading to the production of inflammation-inducing cytokines resulting in immune system overdrive(2). However, UCSF's findings were not reflective within this archetype's network, therefore, it is simply implicated that SarsCov2 has an involvment with cyclins for its own use for viral replication or cell takeover.

2. https://www.ucsf.edu/news/2020/06/417936/covid-19-relies-cells-master-regulators-survival

<img src="arc4.png" width="800" />

Archetype 5

LEF1 induced Wnt signaling activates transcription of epithelial to mesenchymal transition(EMT)(3) involving cell migration, metastasis, and invasion. However, this proclaimed biomarker has only been suggested among cancerous cells. Similar EMT pathways may be suggested in SarsCov2. MYC activates growth related genes and binds to the VEGFA promoter promoting VEGFA production and sprouting angiogenesis, thus allowing permeability in the blood vessel. Given BCL2's function to inhibit apoptosis in nueral cells and lymphocytes, the pathway involving LEF1, MYC, and BCL2 suggest vascular endothelial cell growth resulting in the permeability of infected lymphocytes through blood vessels. Thus, looking into VEGFA and BCL2 inhibitors may combat virus spread in blood vessels. 

The link between LTB and BIRC3 also suggest metastatic properties thus providing further details on cell migration induced by the LTB cell membrane receptor to call in an inflammotory response that may affect a modulator of inflammotory signaling,BIRC3, which may affect cell invasion and metastasis.


3. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5489786/

<img src="arc5.png" width="800" />

Archetype 6

Genes in this archetype involve humoral responses and B cell antigen recoginition, and B-cell differentiation including MZB1,IGF,TNFRSF17, and POU2AF1.

<img src="arc6.png" width="800" />

Multiple archetypes suggest neuronal differentiation from archetype 1, blood clotting from archetype 2, and invasive cell properties from archetype 5. To further analyze archetype 5 that details cell invasion and metastasis pathways, I will perform another Pareto analysis on the cell type distinguished by archetype 5 only. 

# Pareto Archetypal Analysis among CD4 T cells identified at Archetype 5

# R code shown below. 

nearCells = bin_cells_by_arch(mergeDat$data,mergeDat$arc_col,bin_prop = 0.1,return_names = TRUE,dist_cutoff = 0.01)

celltypes <- data.frame(nearCells)

celltypes <- data.frame(covid$cell.type)

celltypes <- data.frame(cells = rownames(celltypes),celltypes,row.names = NULL)

arc5 <- subset(celltypes,cells %in% nearCells$archetype_5)

Idents(object = covid) <- "cell.type"

indCells <- subset(covid,idents = c("CD4m T","CD4n T"))

indCellsPCS <- data.frame(indCells@reductions$pca\@cell.embeddings)

indCellsPCS <- indCellsPCS[c(1:10)]

indCellsPCS <- data.matrix(indCellsPCS)

indCellsPCS <- t(indCellsPCS)

indCellPcha <- k_fit_pch(indCellsPCS,ks = 3:8,maxiter = 500,seed=345)

CD4PCHA <- fit_pch(indCellsPCS, noc = 3,conv_crit= 1e-06,maxiter = 500,volume_ratio = "t_ratio")

plt_arc <- plot_arc(arc_data = CD4PCHA, data = indCellsPCS)

print(plt_arc)

<img src="CD4Arcs.png" width="800" />


obj <- merge_arch_dist(arc_data = CD4PCHA,data = indCellsPCS,dist_metric = c("euclidean","arch_weights")[1],feature_data = indCells@assays$RNA\@scale.data)

CD4Genes <- find_decreasing_wilcox(obj\\$data,obj\\$arc_col,features = obj$features_col,bin = 0.1,method = "BioQC")

CD4Stats <- get_top_decreasing(summary_genes = CD4Genes,summary_sets = NULL, cutoff_genes = 0.05,cutoff_metric = "wilcoxon_p_val",p.adjust.method = "BH")

dt2 <- fwrite(x = CD4Stats$enriched_genes,file="CD4.csv")

In [4]:
gsea = ParetoGSEA(fileObj = 'CD4.csv',nGenes = 25)
gsea.topGenesOnly()
gsea.runGSEA()

archetype_1
                                                          es       nes  \
Term                                                                     
regulation of cell proliferation (GO:0042127)       0.863636  2.908140   
regulation of apoptotic process (GO:0042981)        0.698462  2.822348   
negative regulation of binding (GO:0051100)         1.000000  2.778113   
regulation of DNA binding (GO:0051101)              1.000000  2.736466   
negative regulation of oxidoreductase activity ...  1.000000  2.727704   
regulation of endodeoxyribonuclease activity (G...  1.000000  2.707794   
negative regulation of interleukin-12 productio...  0.956522  2.686859   
cellular protein catabolic process (GO:0044257)     1.000000  2.668523   
positive regulation of MAPK cascade (GO:0043410)    0.818182  2.668099   
regulation of T cell activation (GO:0050863)        0.818182  2.637292   

                                                        pval       fdr  \
Term                     

In [6]:
gsea = ParetoGSEA(fileObj = 'CD4.csv',nGenes = 25)
gsea.topGenesOnly()
gsea.saveGeneRanks()

I observe the overall network consisting of all genes expressed among CD4 T cells for all archetypes. PDK1, LDHA, MYC, and LEF1, and TCF7 form a linear connection. Based on the previous analysis of the role of MYC and LEF1, PDK1 has a role in cell proliferation and reducing apoptosis in response to hypoxia and oxidative stress, and also functions within pathways involving insulin-like growth factor and insulin signaling. PDK1 has been researched to induce epithilial-mesenchymal transition in various cancer metastasis(4,5) and overall cell invasion and migration. 

The role of integrins in this networks including receptors ITGA6, ITGB1, and ITGB2 along with the family of chemokines including CCR7, CCR4, CCR5, CCL5,and CXCR6 correspond to integrin activation by chemokines for T-cell adhesion to endothelial cells creating an outside-in signaling from the extra-cellular matrix to the cytoplasm(6). Causes for this mechanism include  tumor invasion and atherosclerosis. Based on this chemokine activation of T cells on the endothelial surface, it can be implicated that this process is within many different organs, thus contributing to viral spread and perhaps blood clots/viral invasions in different organs. Furthermore, it is plausible for matrix degradation to contribute to the virus spread given T cell migration/adhesion within endothelial cells.

The network involving IFI27, SAMHD1,ISG15,OAS2, and other genes correspond to anti-viral responses.


4. https://www.sciencedirect.com/science/article/abs/pii/S0014482719306299
5. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4745712/
6. https://pubmed.ncbi.nlm.nih.gov/11005242/


<img src="CD4.png" width="800" />

An archetype analysis gives appropriate biological processes among genes found closest to each arch. The biological processes give insight on Sars-Cov-2 role in the immune system that deviate from the expected innate,adaptive immune responses. With this approach, we can determine which cell type is involved in processes describing virus invasion and spread, and thus can gain a dynamic understanding of key processes occuring within this cell type. Further analysis of interest would be pseudo-time analysis within CD4 T cells to observe cell differentiation within a trajectory. 

Key Takeaways:

1. Integrin family proteins are involved in both chemokine mediated T cell migration and platelet aggregation.

2. Proteins including MYC,LEF1,PDK1, and BIRC3 detail mechanisms behind virus spread,virus role in cell protrusion, and mechanisms of epithilial to mesenchymal transition.

3. A small network consisting of proteins SYNE1, SYNE2 and CX3CR1 may contribute to dendritic cell chemotaxis and cerebral cortex migration implicating neuronal pathogenesis.

4. Certain networks suggest a pro-inflammatory respone induced by MAPK and ERK 1/2 signaling that may lead to a cytokine storm, a known effect of Sars-Cov2 from early findings.